In [ ]:
malid = 1

In [ ]:
const medium = "anime";

In [ ]:
using CSV
using DataFrames
using LinearAlgebra
import NBInclude: @nbinclude
using SparseArrays
using StatsBase: cor
using Statistics: mean
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [ ]:
ENV["DATAFRAMES_COLUMNS"] = 300;
ENV["DATAFRAMES_ROWS"] = 300;

In [ ]:
function get_media_df(medium)
    media_to_uid = DataFrame(CSV.File("../../data/processed_data/$(medium)_to_uid.csv"))
    media = DataFrame(
        CSV.File(
            "../../data/processed_data/$(medium).csv",
            ntasks = 1;
            stringtype = String,
        ),
    )
    innerjoin(media_to_uid, media, on = "mediaid" => "$(medium)_id")
end;

In [ ]:
function get_neighborhood(item, W, K; transpose = false)
    weights = W[:, item]
    order = sortperm(weights, rev = true)[1:K]
    order[1:K], weights[order[1:K]]
end;

In [ ]:
function get_series(df, media_to_uid)
    df = copy(df)
    df[!, :uid] = df[:, Symbol(medium)] .- 1
    sim = innerjoin(df, media_to_uid, on = "uid")
    sort(sim, :weight, rev = true)
end;

In [ ]:
function display(df::DataFrame)
    df = select(df, :title, :weight, :uid)
    headers = titlecase.(Base.replace.(names(df), "_" => " "))
    Base.show(
        stdout,
        MIME("text/html"),
        df;
        allow_html_in_cells = true,
        header = headers,
        row_number_column_title = "Rank",
        top_left_str = "",
    )
end;

In [ ]:
function display_related_series(mal_uid, media_to_uid, W)
    uid = only(media_to_uid[media_to_uid[:, "mediaid"].==mal_uid, :uid])
    items, weights = get_neighborhood(uid + 1, W, 100)
    df = DataFrame(weight = weights)
    df[:, medium] = items
    get_series(df, media_to_uid) |> display
end;

In [ ]:
media_df = get_media_df(medium)
sim_weights = read_params("$medium/Nondirectional/WatchSimilarity", true)["S"];

In [ ]:
display_related_series(malid, media_df, sim_weights)